# Adversarial Validation

# Test adversarial validation on a sample of data train set

In [126]:
from xgboost import XGBClassifier
import pandas as pd
import os
import sys
import numpy as np

PATH_PROJECT = open("set_path.txt",'r').readlines()[0]
PATH_UTILITIES = PATH_PROJECT+'/code/utilities'
sys.path.append(PATH_UTILITIES)
os.chdir(PATH_UTILITIES)

import data_loading as DL
import data_preparation_for_models as DP
import predictions_analysis as PA
from download_prediction import download_pred_Xtest

In [127]:
datatrain, Y, var_quant, var_quali, var_quali_to_encode = DL.main_load_data()

In [128]:
datatrain = datatrain.drop(['avatar_id'], axis=1)
datatrain.head()

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy
0,paris,40,dutch,0,467,214,Accar Hotels,Safitel,0,0,0
1,paris,40,dutch,0,708,102,Yin Yang,Ardisson,1,0,0
2,paris,40,dutch,0,853,110,Chillton Worldwide,Tripletree,1,0,0
3,paris,40,dutch,0,207,108,Boss Western,J.Halliday Inn,0,0,1
4,paris,40,dutch,0,415,83,Boss Western,Boss Western,1,0,0


In [129]:
datatest = pd.read_csv("../../data/all_data/test_set_complet.csv")

In [130]:
datatest = datatest.drop(['order_requests', 'avatar_id'], axis=1)
datatest.head()

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy
0,vilnius,21,romanian,0,161,46,Boss Western,J.Halliday Inn,1,0,0
1,vilnius,21,romanian,0,187,32,Accar Hotels,Marcure,1,1,0
2,vilnius,21,romanian,0,279,12,Independant,Independant,1,0,0
3,vilnius,21,romanian,0,395,10,Accar Hotels,Ibas,0,0,0
4,vilnius,21,romanian,0,488,42,Accar Hotels,Safitel,1,1,1


In [131]:
datatrain["is_test"] = 0
datatest["is_test"] = 1

In [132]:
datatrain = datatrain.sample(n=6640)
datatrain.head()

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy,is_test
538486,paris,13,irish,0,742,21,Accar Hotels,Safitel,1,1,0,0
336933,madrid,35,french,0,828,35,Independant,Independant,0,0,0,0
954503,vilnius,13,dutch,0,89,9,Independant,Independant,0,0,0,0
956487,paris,8,czech,1,69,66,Accar Hotels,Safitel,0,0,0,0
249335,paris,8,italian,1,133,0,Independant,Independant,0,0,0,0


In [133]:
datatest.head()

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy,is_test
0,vilnius,21,romanian,0,161,46,Boss Western,J.Halliday Inn,1,0,0,1
1,vilnius,21,romanian,0,187,32,Accar Hotels,Marcure,1,1,0,1
2,vilnius,21,romanian,0,279,12,Independant,Independant,1,0,0,1
3,vilnius,21,romanian,0,395,10,Accar Hotels,Ibas,0,0,0,1
4,vilnius,21,romanian,0,488,42,Accar Hotels,Safitel,1,1,1,1


In [134]:
frames = [datatrain, datatest]
data = pd.concat(frames,axis=0)

In [135]:
data_X = data.drop(["is_test"], axis=1)
data_Y = pd.DataFrame(data.is_test)

In [136]:
data_X = data_X.drop(["city", "language", "hotel_id", "group", "brand"], axis=1)

In [137]:
data_Y

,is_test
538486,0
336933,0
954503,0
956487,0
249335,0
...,...
6639,1
6640,1
6641,1
6642,1


## Dummy variables

In [138]:
# data_X = pd.get_dummies(data_X, columns=["city", "language", "hotel_id", "group", "brand"])

In [139]:
data_X.head()

,date,mobile,stock,parking,pool,children_policy
538486,13,0,21,1,1,0
336933,35,0,35,0,0,0
954503,13,0,9,0,0,0
956487,8,1,66,0,0,0
249335,8,1,0,0,0,0


In [140]:
# fit model no training data
model = XGBClassifier()
model.fit(data_X, data_Y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [141]:
# make predictions
y_pred = model.predict(data_X)
predictions = [round(value) for value in y_pred]

In [142]:
# evaluate predictions
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(data_Y, predictions)
accuracy = auc(fpr, tpr)
print("AUC: %.2f%%" % (accuracy * 100.0))

AUC: 85.60%


In [143]:
pd.DataFrame(y_pred)

,0
0,0
1,1
2,0
3,0
4,0
...,...
13279,1
13280,1
13281,1
13282,1


In [144]:
predVSreal = pd.DataFrame({'predictions': y_pred,
                          'real': data_Y.to_numpy().reshape(-1)})


In [145]:
# indexes of false discovery
idxs_of_fd = predVSreal.index[(predVSreal.predictions==1) & (predVSreal.real==0)]

# Script to select train data acccording to adversarial validation

In [146]:

import data_loading as DL
import data_preparation_for_models as DP
import predictions_analysis as PA
from download_prediction import download_pred_Xtest

datatest = pd.read_csv("../../data/all_data/test_set_complet.csv")
datatest = datatest.drop(['order_requests', 'avatar_id'], axis=1)
datatest["is_test"] = 1

# init dataset and batches
datatrain, Y, var_quant, var_quali, var_quali_to_encode = DL.main_load_data()
Ybis = pd.DataFrame(dict({'price':Y}))
#datatrain = datatrain.drop(['avatar_id'], axis=1)
datatrain["is_test"] = 0
datatrain = pd.concat([datatrain, Ybis], axis=1)

datatrain_batch = np.array_split(datatrain, 100)
fd_dataframe = pd.DataFrame([])
estimated_accuracy = 0

stock_prices = pd.DataFrame([])

# to iterate
for i in range(100):
    # get batch and fit classifier model
    train_sample = datatrain_batch[i].drop(['price','avatar_id'], axis=1)
    frames = [train_sample, datatest]
    data = pd.concat(frames,axis=0)
    data_X = data.drop(["is_test"], axis=1)
    data_Y = pd.DataFrame(data.is_test)
    data_X = data_X.drop(["city", "language", "hotel_id", "group", "brand"], axis=1)
    model = XGBClassifier()
    model.fit(data_X, data_Y)

    # make predictions
    y_pred = model.predict(data_X)
    predictions = [round(value) for value in y_pred]
    predVSreal = pd.DataFrame({'predictions': y_pred,
                              'real': data_Y.to_numpy().reshape(-1)})
    
    # update accuracy
    fpr, tpr, thresholds = roc_curve(data_Y, predictions)
    accuracy = auc(fpr, tpr)
    estimated_accuracy += accuracy
    # get False Discoveries (predicted test, were train)
    idxs_of_fd = predVSreal.index[(predVSreal.predictions==1) & (predVSreal.real==0)].to_list()
    fd_list = train_sample.iloc[idxs_of_fd,]
    stock_prices = pd.concat((stock_prices, datatrain_batch[i].iloc[idxs_of_fd,]))
    fd_dataframe = pd.concat((fd_dataframe, fd_list), axis=0)
    
print("estimated accuracy of train/test classification (Monte Carlo): ", estimated_accuracy/100)

estimated accuracy of train/test classification (Monte Carlo):  0.8747720521710423


In [147]:
adversarial_data = fd_dataframe.join(Ybis)

In [148]:
adversarial_data

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy,is_test,price
2,paris,40,dutch,0,853,110,Chillton Worldwide,Tripletree,1,0,0,0,147
9,paris,40,dutch,0,631,43,Yin Yang,8 Premium,1,0,0,0,81
14,paris,40,dutch,0,564,42,Accar Hotels,Ibas,0,0,0,0,92
17,paris,40,dutch,0,359,86,Accar Hotels,Navatel,0,0,0,0,111
23,paris,40,dutch,0,903,88,Independant,Independant,1,0,0,0,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...
960667,vilnius,17,lithuanian,1,819,42,Yin Yang,Royal Lotus,0,0,0,0,119
960692,vilnius,17,lithuanian,1,498,9,Boss Western,Boss Western,0,0,0,0,101
960699,vilnius,17,lithuanian,1,47,23,Accar Hotels,Marcure,1,0,0,0,152
960702,vilnius,17,lithuanian,1,451,16,Independant,Independant,1,0,0,0,109


# Save selected data to csv

In [ ]:
adversarial_data.to_csv("../../data/adversarial_validation_data/selected_train_data.csv")